<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/asosMetar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## From https://mesonet.agron.iastate.edu/request/download.phtml?network=CO_ASOS
## https://github.com/akrherz/iem/blob/main/scripts/asos/iem_scraper_example.py

In [1]:
import json
import time
import datetime
from urllib.request import urlopen
import pandas as pd

In [2]:
# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"

In [3]:
def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""

In [4]:
def get_stations_from_filelist(filename):
    """Build a listing of stations from a simple file listing the stations.
    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return 

In [5]:
def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = []
    #states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
    #MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
    #WA WI WV WY"""
    states = """MD DC""" # my states option
    networks = []
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = (
            "https://mesonet.agron.iastate.edu/geojson/network/%s.geojson"
        ) % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict["features"]:
            stations.append(site["properties"]["sid"])
    return stations

In [ ]:
def download_alldata():
    """An alternative method that fetches all available data.
    Service supports up to 24 hours worth of data at a time."""
    # timestamps in UTC to request data for
    startts = datetime.datetime(2012, 8, 1)
    endts = datetime.datetime(2012, 9, 1)
    interval = datetime.timedelta(hours=24)

    service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

    now = startts
    while now < endts:
        thisurl = service
        thisurl += now.strftime("year1=%Y&month1=%m&day1=%d&")
        thisurl += (now + interval).strftime("year2=%Y&month2=%m&day2=%d&")
        print("Downloading: %s" % (now,))
        data = download_data(thisurl)
        outfn = "%s.txt" % (now.strftime("%Y%m%d"),)
        with open(outfn, "w") as fh:
            fh.write(data)
        now += interval

In [26]:
# timestamps in UTC to request data for
startts = datetime.datetime(2022, 5, 7)
endts = datetime.datetime(2022, 5, 9)

In [27]:
service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
service += endts.strftime("year2=%Y&month2=%m&day2=%d&")

# Two examples of how to specify a list of stations
# stations = get_stations_from_networks()

In [28]:
service

'http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2022&month1=05&day1=07&year2=2022&month2=05&day2=09&'

In [29]:
stations = ['KBWI']

In [30]:
stations

['KBWI']

In [31]:
    for station in stations:
        uri = "%s&station=%s" % (service, station)
        print("Downloading: %s" % (station,))
        data = download_data(uri)
        outfn = "%s_%s_%s.txt" % (
            station,
            startts.strftime("%Y%m%d%H%M"),
            endts.strftime("%Y%m%d%H%M"),
        )
        out = open(outfn, "w")
        out.write(data)
        out.close()

Downloading: KBWI


In [12]:
#data

In [32]:
outfn

'KBWI_202205070000_202205090000.txt'

In [23]:
out

<_io.TextIOWrapper name='KBWI_202205070000_202205090000.txt' mode='w' encoding='UTF-8'>

In [33]:
f2read = outfn

In [41]:
with open(f2read, "r") as file:
   # FileContent = file.read()
   # print(FileContent)

   for i in range(10):
        line = next(file).strip()
        print(line)

#DEBUG: Format Typ    -> comma
#DEBUG: Time Period   -> 2022-05-07 00:00:00+00:00 2022-05-09 00:00:00+00:00
#DEBUG: Time Zone     -> Etc/UTC
#DEBUG: Data Contact   -> daryl herzmann akrherz@iastate.edu 515-294-5978
#DEBUG: Entries Found -> -1
station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
BWI,2022-05-07 00:00,-76.6683,39.1754,M,M,M,70.00,11.00,0.01,29.70,M,2.00,M,OVC,M,M,M,600.00,M,M,M,-RA BR,M,M,M,M,M,M,M,KBWI 070000Z AUTO 07011KT 2SM -RA BR OVC006 14/12 A2970 RMK P0001 T01400120  LTG DSNT SW! MADISHF,M
BWI,2022-05-07 00:05,-76.6683,39.1754,M,M,M,70.00,14.00,0.02,29.70,M,2.00,21.00,OVC,M,M,M,600.00,M,M,M,-RA BR,M,M,M,M,M,M,M,KBWI 070005Z AUTO 07014G21KT 2SM -RA BR OVC006 14/12 A2970 RMK P0002 T01400120 MADISHF,M
BWI,2022-05-07 00:10,-76.6683,39.1754,M,M,M,80.00,14.00,0.05,29.70,M,2.00,19

In [43]:
testData = pd.read_csv(f2read, skiprows=5 )

In [45]:
#testData.head(2)

In [46]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station            639 non-null    object 
 1   valid              639 non-null    object 
 2   lon                639 non-null    float64
 3   lat                639 non-null    float64
 4   tmpf               639 non-null    object 
 5   dwpf               639 non-null    object 
 6   relh               639 non-null    object 
 7   drct               639 non-null    object 
 8   sknt               639 non-null    object 
 9   p01i               639 non-null    object 
 10  alti               639 non-null    float64
 11  mslp               639 non-null    object 
 12  vsby               639 non-null    float64
 13  gust               639 non-null    object 
 14  skyc1              639 non-null    object 
 15  skyc2              639 non-null    object 
 16  skyc3              639 non

In [47]:
testData.T

,0,1,2,3,4,5,6,7,8,9,...,629,630,631,632,633,634,635,636,637,638
station,BWI,BWI,BWI,BWI,BWI,BWI,BWI,BWI,BWI,BWI,...,BWI,BWI,BWI,BWI,BWI,BWI,BWI,BWI,BWI,BWI
valid,2022-05-07 00:00,2022-05-07 00:05,2022-05-07 00:10,2022-05-07 00:15,2022-05-07 00:20,2022-05-07 00:25,2022-05-07 00:30,2022-05-07 00:30,2022-05-07 00:35,2022-05-07 00:40,...,2022-05-08 23:15,2022-05-08 23:20,2022-05-08 23:25,2022-05-08 23:30,2022-05-08 23:35,2022-05-08 23:40,2022-05-08 23:45,2022-05-08 23:50,2022-05-08 23:54,2022-05-08 23:55
lon,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,...,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683,-76.6683
lat,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,...,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754,39.1754
tmpf,M,M,M,M,M,M,55.90,M,M,M,...,M,M,M,M,M,M,M,M,55.90,M
dwpf,M,M,M,M,M,M,54.00,M,M,M,...,M,M,M,M,M,M,M,M,34.00,M
relh,M,M,M,M,M,M,93.32,M,M,M,...,M,M,M,M,M,M,M,M,43.46,M
drct,70.00,70.00,80.00,80.00,60.00,70.00,70.00,70.00,60.00,70.00,...,80.00,70.00,70.00,80.00,80.00,70.00,80.00,80.00,70.00,70.00
sknt,11.00,14.00,14.00,14.00,14.00,15.00,13.00,13.00,13.00,13.00,...,8.00,5.00,8.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
p01i,0.01,0.02,0.05,0.11,0.13,0.15,0.15,0.15,0.16,0.16,...,M,M,M,M,M,M,M,M,0.00,M
